# Setup

In [ ]:
%pip install astropy
%pip install git+https://github.com/pvigier/perlin-numpy
%pip install tensorflow_addons
%pip install wandb

In [ ]:
!git clone https://github.com/tomasmuzas/phd

In [1]:
from phd.Augmentations.common_code.augmentation_functions import *
from phd.Augmentations.common_code.get_dataset import *
from phd.Augmentations.common_code.models import *
from phd.Augmentations.common_code.training_loop import *
from phd.Augmentations.common_code.utils import *
import tensorflow as tf

tf.get_logger().setLevel('ERROR')

In [2]:
from google.colab import auth

auth.authenticate_user()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Running on GPU.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
!apt-get update -y
!curl -LJO "https://github.com/GoogleCloudPlatform/gcsfuse/releases/download/v1.2.0/gcsfuse_1.2.0_amd64.deb"
!apt-get -y install fuse
!apt-get clean
!dpkg -i "gcsfuse_1.2.0_amd64.deb"
!mkdir gcs
!gcsfuse --implicit-dirs tomasmuzas-phd-datasets gcs

In [ ]:
import wandb

wandb.login()

# Training

In [ ]:
tf.get_logger().setLevel('ERROR')

for augmentations in [False, True]:
  for folds, train_size, test_size in [("90-10", 648192, 67817) , ("80-20", 575488, 143827 ), ("50-50", 360448, 359567), ("20-80", 144384, 575306), ("10-90", 72704, 647220)]:
    print(f"augmentations: {augmentations}, setup: {folds}")
    training_config = {
    "ENABLE_DETERMINISM": False,
    "SEED": 17,
    "AUTO": tf.data.AUTOTUNE,
    "TRAIN_BATCH_SIZE": 1024,
    "TEST_BATCH_SIZE": 128,
    "NUMBER_OF_EPOCHS": 500,
    "EARLY_STOPPING_TOLERANCE": 10,
    "LEARNING_RATE": 1e-4,
    "IMAGE_SIZE": 128,
    "SHUFFLE_BUFFER": train_size,
    "TPU": True,
    "AUGMENTATIONS_ROTATE": augmentations,
    "AUGMENTATIONS_FLIP_HORIZONTALLY": augmentations,
    "AUGMENTATIONS_FLIP_VERTICALLY": augmentations,
    "AUGMENTATIONS_ZOOM": augmentations,
    "AUGMENTATIONS_RANDOM_NOISE": augmentations,
    "AUGMENTATIONS_CENTER_NOISE": False,
    "AUGMENTATIONS_OUTSIDE_NOISE": False,
    "AUGMENTATIONS_PERLIN_CENTER_NOISE": False,
    "USE_ADABELIEF_OPTIMIZER": False,
    "GALAXY_DATASET": "SDSS",
    "TRAIN_DATASET_SIZE": train_size,
    "TEST_DATASET_SIZE": test_size,
    "STRETCH": False,
    "STEPS_PER_EXECUTION": 10,
    "FOLDS": 1,
    "NUMBER_OF_CLASSES": 5
    }
    training_config["WANDB_PROJECT_NAME"] = f"Spiral Certainty Bins, {training_config['NUMBER_OF_CLASSES']} classes, {folds}, {'no' if not augmentations else ''} augmentations"
    training_config["EXPERIMENT_DESCRIPTION"] = f"spiral_certainty_bins_{'no_' if not augmentations else ''}augmentations_{training_config['NUMBER_OF_CLASSES']}_classes_{folds}"
    # training_config["LOCAL_GCP_PATH_BASE"] = f"../../../../Step 2 (Bins) - Image dataset/"
    # training_config["REMOTE_GCP_PATH_BASE"] = f"../../../../Step 2 (Bins) - Image dataset/"
    # training_config["DATASET_PATH"] = f"TFDataset/{training_config['FOLDS']}_folds"
    training_config["LOCAL_GCP_PATH_BASE"] = f"gcs/SpiralCertaintyBin"
    training_config["REMOTE_GCP_PATH_BASE"] = f"gs://tomasmuzas-phd-datasets/SpiralCertaintyBin"
    training_config["DATASET_PATH"] = f"data/TFDataset/MCCV/5_Classes/{folds}"

    models = [
        {'name': 'cavanagh', 'func': Cavanagh, 'starting_fold': 1},
        {'name': 'dieleman', 'func': Dieleman, 'starting_fold': 1},
        {'name': 'resnet50', 'func': create_ResNet50, 'starting_fold': 1},
        # {'name': 'newsimple', 'func': SimpleModel, 'starting_fold': 1}
    ]

    perform_training(models, training_config)